In [16]:
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device('cuda') # 'cpu' or 'cuda'
print(torch.__version__)

2.0.1+cu118


In [17]:
## model parameters
n=16 #150
m=16 #96
snr = 20.0  # SNR per receive antenna [dB]
##
N=2*n
M=2*m
##

## parameters for evauation of generalization error
bs = 10000 # number of samples of x for each H
ss = 1000 # number of H
##

# SB settings
eps = 1.0
T_max = 50
pump_SB = 1.0/(T_max*eps) #0.01 # pump coeff
D_SB = 1. # delta
xi_SB = 0.1 # xi_0


In [18]:
def x_gen(bs,n):
    x = torch.rand(bs,n).to(device)
    x[x<0.5] = -1
    x[x>0.5] = 1
    return x
def y_gen(bs,m,x0,H,sigma_std):
    return x0@H+ torch.normal(0.0, sigma_std*torch.ones(bs, m)).to(device)

def trans_2_QUBO(H,y):
    J = H@H.t() - torch.diag(torch.diagonal(H@H.t(),0))
    h = -2*y@H.t()
    return J,h

def trans_2_QUBO_LMMSE(H,y,lam):
    H_inv = torch.linalg.inv(H.t()@H+lam*torch.eye(M,device=device)) #dim:M*M
    J = H@H_inv@H.t() - torch.diag(torch.diagonal(H@H_inv@H.t(),0))
    h = -2*y@H_inv@H.t()
    lmax_2 = ((J*J).sum()/(N*(N-1)))**0.5 #estimated max. eig.
    return J,h, 1.0/(2*N**0.5*lmax_2)

def BER(x,y):
    z = torch.ones(x.size()).to(device)
    z[torch.isclose(torch.sign(x),torch.sign(y))] = 0.
    return z.sum()/(z.numel())

seed_ =12
torch.manual_seed(seed_)
# QPSK
def H_gen(m,n):
    H_re = torch.normal(0.0, std=math.sqrt(0.5) * torch.ones(n,m))
    H_im = torch.normal(0.0, std=math.sqrt(0.5) * torch.ones(n,m))  # sensing matrix
    H = torch.cat((torch.cat((H_re,H_im),0),torch.cat((-1*H_im,H_re),0)),1)
    H = H.to(device)
    return H

#SNR
def est_SNR(snr,m,n):
    sigma2 = (2*n/math.pow(10,snr/10.0))/2.0
    sigma_std = math.sqrt(sigma2)
    return sigma_std


### ML-SB
naive SB detector maximum log likelihood

In [19]:
# pumping amp.
def Pump(t,pump_SB):
    #print(t, t/(T_max*eps))
    return  (t/(T_max*eps))**1.0 #pump_SB * t

def Dqd(q, t,J,h,D_SB,pump_SB,xi_SB,bai,d_flag):
    if d_flag == 0:
        # ballistic ver. by arXiv:2210.14660
        DE_QUBO = q@J + 0.5*h
    if d_flag == 1:
        # discritized ver. by arXiv:2210.14660
        DE_QUBO = q.sign()@J + 0.5*h
    return - bai*(-Pump(t,pump_SB) + D_SB) * q - xi_SB * DE_QUBO

def dSB(q,p,t,eps,J,h,D_SB,pump_SB,xi_SB,bai,d_flag):
    q_ = q + (eps * D_SB) * p #diff(p,K)
    q_2 = torch.clamp(q_, min=-1.,max=1.)
    p_ = p + eps * Dqd(q_2,t,J,h,D_SB,pump_SB,xi_SB,bai,d_flag) #diff(q,Po)
    p_[torch.abs(q_)>1] = 0.0
    return q_2,p_

def dSB_MIMO(T_max,bs,M,N,J,h,D_SB,pump_SB,xi_SB,eps,bai,d_flag):
    q = torch.zeros(bs, N,device=device) # x
    p = torch.zeros(bs, N,device=device) # y
    q_traj = np.zeros([T_max, N]) # trajectory
    p_traj = np.zeros([T_max, N]) # trajectory
    p = torch.randn(bs,N,device=device)
    #p[:,:] = torch.randn(bs,N)#0.2#1.0/N**2
    #ene_init = ene(q, p, np.zeros(1)).item()
    #print(p)
    t = 0.0

    for i in range(T_max):
        t = t + eps
        q, p = dSB(q,p,t,eps,J,h,D_SB,pump_SB,xi_SB,bai,d_flag)
        q_traj[i]=q[0,:].cpu().detach().numpy()
        p_traj[i]=p[0,:].cpu().detach().numpy()
    return q, p, q_traj, p_traj

In [20]:
trial=1 # repeating from another init point
ss = 1000
d_flag = 0 #0=ballisctic(no sign), 1=digital(sign)

print("SNR, BER")
for snr in np.arange(5.0,35.1,5.0):
    sigma_std = est_SNR(snr, m,n)
    ber_ =0.0
    for i in range(ss):
        H = H_gen(m,n)
        sol = x_gen(bs,N)
        y = y_gen(bs,M,sol,H,sigma_std)
        J, h = trans_2_QUBO(H,y)
        lmax_2 = ((J*J).sum()/(N*(N-1)))**0.5 #estimated max. eig.
        #print(torch.linalg.eigvals(J.t()@J).abs().max()**0.5,torch.linalg.eigvals(J_.t()@J_).abs().max()**0.5,2*N**0.5*lmax_2)
        xi_SB_ = D_SB/(2*N**0.5*lmax_2)
        xx = torch.zeros(bs,N,device=device)
        res = 100*torch.ones(bs,N,device=device)
        for k in range(trial):
            x_hat ,_,q_traj ,_= dSB_MIMO(T_max,bs,M,N,J,h,D_SB,pump_SB,xi_SB_,eps,1.0,d_flag)
            res_ = (y-x_hat.sign()@H).norm(dim=1).view(bs,1).repeat(1,N).view(bs,N)
            #print(res_)
            xx[res_<res] = x_hat[res_<res]
            res[res_<res] = res_[res_<res]
        ber_ += BER(sol,xx.sign())
    print(snr, ber_.item()/ss)

SNR, BER
5.0 0.10896620178222656
10.0 0.010583418846130371
15.0 0.0013047537803649902
20.0 0.0006992751955986023
25.0 0.000622352659702301
30.0 0.0005300716757774353
35.0 0.000576202630996704


### MMSE-guided SB

cf) W.Zhang and Y-L. Zheng, arXiv:2210.14660, 2022

In [21]:
# pumping amp.
def Pump(t,pump_SB):
    #print(t, t/(T_max*eps))
    return  (t/(T_max*eps))**1.0 #pump_SB * t

def Dqd2(q, t,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse):
    if d_flag == 0:
        # ballistic ver. by arXiv:2210.14660
        DE_QUBO = q@J + 0.5*h+ 0.5*(q - x_lmmse)
    if d_flag == 1:
        # discritized ver. by arXiv:2210.14660
        DE_QUBO = q.sign()@J + 0.5*h+ 0.5*(q - x_lmmse)
    #return - xi_SB * DE_QUBO
    return - bai*(-Pump(t,pump_SB) + D_SB) * q - xi_SB * DE_QUBO

def dSB2(q,p,t,eps,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse):
    q_ = q + (eps * D_SB) * p
    q_2 = torch.clamp(q_, min=-1.,max=1.)
    p_ = p + eps * Dqd2(q_2,t,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse) #diff(q,Po)
    p_[torch.abs(q_)>1] = 0.0
    return q_2,p_

def dSB_MIMOmod(T_max,bs,M,N,J,h,D_SB,pump_SB,xi_SB,eps,bai,d_flag,x_lmmse):
    q = torch.zeros(bs, N,device=device) # x
    p = torch.zeros(bs, N,device=device) # y
    q_traj = np.zeros([T_max, N]) # trajectory
    p_traj = np.zeros([T_max, N]) # trajectory
    p = torch.randn(bs,N,device=device)
    t = 0.0

    for i in range(T_max):
        t = t + eps
        q, p = dSB2(q,p,t,eps,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse)
        q_traj[i]=q[0,:].cpu().detach().numpy()
        p_traj[i]=p[0,:].cpu().detach().numpy()
    return q, p, q_traj, p_traj




In [22]:
##
eps = 1.0
D_SB = 1.0
T_max = 50
trial=1 # repeating from another init point
d_flag = 0 #0=ballisctic(no sign), 1=digital(sign)

print("SNR, BER")
for snr in np.arange(5.0,35.1,5.0):#35.1,5.0):
    sigma_std = est_SNR(snr, m,n)
    ber_ =0.0

    for i in range(ss):
        H = H_gen(m,n)
        sol = x_gen(bs,N)
        y = y_gen(bs,M,sol,H,sigma_std)
        J, h = trans_2_QUBO(H,y)
        #J_, _ = trans_2_QUBO(H,y)
        lmax_2 = ((J*J).sum()/(N*(N-1)))**0.5 #estimated max. eig.
        xi_SB_ = D_SB/(2*N**0.5*lmax_2)
        x_lmmse =y@(torch.linalg.inv(H.t()@H+sigma_std * torch.eye(2*n,device=device))@H.t())
        xx = torch.zeros(bs,N,device=device)
        res = 100*torch.ones(bs,N,device=device)
        for k in range(trial):
            x_hat ,_,q_traj ,_= dSB_MIMOmod(T_max,bs,M,N,J,h,D_SB,pump_SB,xi_SB_,eps,1.0,d_flag,x_lmmse)
            res_ = (y-x_hat.sign()@H).norm(dim=1).view(bs,1).repeat(1,N).view(bs,N)
            #print(res_)
            xx[res_<res] = x_hat[res_<res]
            res[res_<res] = res_[res_<res]
        ber_ += BER(sol,xx.sign())
    print(snr, ber_.item()/ss)

SNR, BER
5.0 0.10828033447265625
10.0 0.009600574493408204
15.0 0.0006834000945091247
20.0 0.0002881717681884766
25.0 0.00020238131284713745
30.0 0.00015670613944530486
35.0 0.00010458118468523025


### LM-SB (LMMSE-like matrix based)

In [23]:
# pumping amp.
def Pump(t,pump_SB):
    #print(t, t/(T_max*eps))
    return  (t/(T_max*eps))**1.0 #pump_SB * t

def Dqd3(q, t,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse):
    if d_flag == 0:
        # ballistic ver. by arXiv:2210.14660
        DE_QUBO = q@J + 0.5*h
    if d_flag == 1:
        # discritized ver. by arXiv:2210.14660
        DE_QUBO = q.sign()@J + 0.5*h
    #return - xi_SB * DE_QUBO
    return - bai*(-Pump(t,pump_SB) + D_SB) * q - xi_SB * DE_QUBO

def dSB2(q,p,t,eps,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse):
    q_ = q + (eps * D_SB) * p
    q_2 = torch.clamp(q_, min=-1.,max=1.)
    p_ = p + eps * Dqd3(q_2,t,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse) #diff(q,Po)
    p_[torch.abs(q_)>1] = 0.0
    return q_2,p_

def dSB_MIMOmod(T_max,bs,M,N,J,h,D_SB,pump_SB,xi_SB,eps,bai,d_flag,x_lmmse):
    q = torch.zeros(bs, N,device=device) # x
    p = torch.zeros(bs, N,device=device) # y
    q_traj = np.zeros([T_max, N]) # trajectory
    p_traj = np.zeros([T_max, N]) # trajectory
    p = torch.randn(bs,N,device=device)
    t = 0.0

    for i in range(T_max):
        t = t + eps
        q, p = dSB2(q,p,t,eps,J,h,D_SB,pump_SB,xi_SB,bai,d_flag,x_lmmse)
        q_traj[i]=q[0,:].cpu().detach().numpy()
        p_traj[i]=p[0,:].cpu().detach().numpy()
    return q, p, q_traj, p_traj




In [25]:
def trans_2_QUBO2(H,y):
    J = H@H.t() - torch.diag(torch.diagonal(H@H.t(),0))
    h = -2*y@H.t()
    return J,h

##
LM_lam = 1.0 #lambda in LMMSE matrix
T_max = 50
trial=1 # repeating from another init point
ss = 1000
d_flag = 0 #0=ballisctic(no sign), 1=digital(sign)


print("SNR, BER")
for snr in np.arange(5.0,35.1,5.0):#35.1,5.0):
    sigma_std = est_SNR(snr, m,n)
    ber_ =0.0
    for i in range(ss):
        H = H_gen(m,n)
        sol = x_gen(bs,N)
        y = y_gen(bs,M,sol,H,sigma_std)
        J, h, _ = trans_2_QUBO_LMMSE(H,y,LM_lam)
        lmax_2 = ((J*J).sum()/(N*(N-1)))**0.5 #estimated max. eig.
        xi_SB_ = D_SB/(2*N**0.5*lmax_2)

        xx = torch.zeros(bs,N,device=device)
        res = 100*torch.ones(bs,N,device=device)
        for k in range(trial):
            x_hat ,_,q_traj ,_= dSB_MIMOmod(T_max,bs,M,N,J,h,D_SB,pump_SB,xi_SB_,eps,1.0,d_flag,x_lmmse)
            res_ = (y-x_hat.sign()@H).norm(dim=1).view(bs,1).repeat(1,N).view(bs,N)
            #print(res_)
            xx[res_<res] = x_hat[res_<res]
            res[res_<res] = res_[res_<res]
        ber_ += BER(sol,xx.sign())
    print(snr,ber_.item()/ss)

SNR, BER
5.0 0.17868038940429687
10.0 0.05032455444335938
15.0 0.001269072413444519
20.0 3.718754043802619e-06
25.0 1.4687497168779373e-06
30.0 7.437500171363354e-07
35.0 7.249999907799065e-07
